In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import output_notebook, figure, show, ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

# Precio del dólar mensual en pesos mexicanos.

In [2]:
#dolar = quandl.get("FED/RXI_N_M_MX")
dolar = pd.read_csv("dolar_m", index_col=0, parse_dates=True) # interpreta 1a col. como fechas, no como cadenas
print(dolar[:10])
p=figure(plot_width=600, plot_height=300, 
         x_axis_type="datetime",
         tools="hover,pan,wheel_zoom")
p.line(dolar.index, dolar["Value"].values)
hover = p.select(dict(type=HoverTool))
hover.tooltips={"índice":"$index", "fecha":"@x{%F}" ,"valor":"$@y"}
hover.formatters = { "x": "datetime"}
hover.mode="vline"
show(p)

             Value
Date              
1993-11-30  3.1498
1993-12-31  3.1083
1994-01-31  3.1078
1994-02-28  3.1218
1994-03-31  3.3026
1994-04-30  3.3495
1994-05-31  3.3167
1994-06-30  3.3671
1994-07-31  3.4031
1994-08-31  3.3814


In [3]:
# Otra forma en Bokeh...
dolar = pd.read_csv("dolar_m", index_col=0, parse_dates=True) # interpreta 1a col. como fechas, no como cadenas
print(dolar[:10])
hover = HoverTool(tooltips = {"índice":"$index", "fecha":"@x{%F}" ,"valor":"$@y"},
                  formatters = { "x": "datetime"},
                  mode="vline")

p=figure(plot_width=600, plot_height=300, 
         x_axis_type="datetime",
         tools=[hover,"pan,wheel_zoom"])

datos = ColumnDataSource(data=dict(
    x = dolar.index,
    y = dolar["Value"].values
))

p.line("x", "y", source=datos)
show(p)

             Value
Date              
1993-11-30  3.1498
1993-12-31  3.1083
1994-01-31  3.1078
1994-02-28  3.1218
1994-03-31  3.3026
1994-04-30  3.3495
1994-05-31  3.3167
1994-06-30  3.3671
1994-07-31  3.4031
1994-08-31  3.3814


In [4]:
#dolar.to_csv("dolar_m")

In [5]:
from bokeh.models.widgets import Slider
from bokeh.models import CustomJS
from bokeh.layouts import column, widgetbox
from bokeh.plotting import curdoc
from bokeh.io import show, output_notebook
output_file("")
#print(dolar["2011-1":"2011-7"])  # si fueran cadenas sería "2011-01":"2011-07"

p=figure()
datos = ColumnDataSource(data=dict(
    x = dolar.index,
    y = dolar["Value"].values
))

inicio = Slider(title="inicio", value=0, start=0, end=278, step=1)
longitud = Slider(title="longitud", value=10, start=5, end=100, step=1)

p.line("x", "y", source=datos)

def actualiza_datos(source=datos, slider=inicio, window=None):
    i = inicio.value
    l = longitud.value
    x = dolar.index[i:i+l]
    y = dolar["Value"].values[i:i+l]
    datos=source
    datos.data = dict(x=x, y=y)
    datos.data.trigger('change')

for w in [inicio, longitud]:
    w.js_on_change('value', CustomJS.from_py_func(actualiza_datos))

inputs = widgetbox(inicio, longitud)
layout = column(inputs, p, width=800)
#print(datos.data['x'])
show(layout)

NameError: name 'output_file' is not defined

interpolando precio del dolar en febrero de 2011...

In [6]:
a = [1,2,3]
a[0]

dolar.loc["2011-1"].values[0]

array([12.128])

In [7]:
enero = dolar["2011-1":"2011-1"].values[0]
marzo = dolar.loc["2011-3"].values[0]
interp = (enero+marzo)/2
print("interpolado:", interp, "real:", dolar.loc["2011-2"].values[0])

interpolado: [12.06215] real: [12.0649]


# Nivel del mar promedio (hemisferio norte)

In [8]:
#heminorte = pd.read_table("http://sealevel.colorado.edu/files/current/sl_nh.txt", 
#                          sep="\s+", index_col=0, header=0, usecols=[0,1])["msl_ib(mm)"]
heminorte = pd.read_csv("heminorte", index_col=0, header=None, names=["year", "msl_ib(mm)"])["msl_ib(mm)"]
#heminorte.to_csv("heminorte")
#heminorte.plot()
heminorte


year
1992.9323    13.717
1992.9595     1.328
1992.9866   -13.375
1993.0138   -24.723
1993.0409   -29.231
1993.0681   -32.538
1993.0952   -35.588
1993.1223   -26.579
1993.1495   -29.622
1993.1766   -34.527
1993.2038   -34.250
1993.2309   -28.518
1993.2581   -30.437
1993.2852   -21.371
1993.3124   -19.196
1993.3395   -23.758
1993.3667   -23.628
1993.3938   -17.808
1993.4210   -10.085
1993.4481    -2.120
1993.4753    -1.728
1993.5024    -4.942
1993.5296     5.074
1993.5567     0.386
1993.5839    22.555
1993.6110    22.201
1993.6382    29.060
1993.6653    23.325
1993.6925    16.635
1993.7196    21.488
              ...  
2015.7639    99.108
2015.7910    96.789
2015.8182    92.405
2015.8453    81.278
2015.8725    79.341
2015.8996    78.108
2015.9268    76.599
2015.9539    69.691
2015.9811    59.957
2016.0082    46.712
2016.0354    42.882
2016.0625    45.051
2016.0897    43.793
2016.1168    46.603
2016.1440    49.218
2016.1711    44.274
2016.1983    33.961
2016.2254    42.199
2016.2526    45

In [35]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries, win=37):
    
    # mMedidas móviles
    rol = timeseries.rolling(window=win, center=True)
    rolmean = rol.mean()
    rolstd = rol.std()

    #graficar medidas móviles:
    p = figure(width=600, height=300,
               title='Media y Desviación Estándard móviles')
    orig = p.line(timeseries.index, timeseries.values, color='blue', muted_alpha=0.2, legend='Original')
    mean = p.line(rolmean.index, rolmean.values, color='red', muted_alpha=0.2, legend='Media móvil')
    std = p.line(rolstd.index, rolstd.values, color='black', muted_alpha=0.2, legend = 'Desviación móvil')
    p.legend.location = "top_left"
    p.legend.click_policy="mute"
    show(p)
    
    #Realiza prueba de Dickey-Fuller:
    print ('Resultados de la prueba de Dickey-Fuller:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Estadística de la prueba','valor-p','#retrasos usados','Número de observaciones'])
    for key,value in dftest[4].items():
        dfoutput['Valor crítico (%s)'%key] = value
    print (dfoutput)

test_stationarity(heminorte)

Resultados de la prueba de Dickey-Fuller:
Estadística de la prueba     -1.474823
valor-p                       0.545927
#retrasos usados             21.000000
Número de observaciones     847.000000
Valor crítico (1%)           -3.438094
Valor crítico (5%)           -2.864958
Valor crítico (10%)          -2.568590
dtype: float64


In [18]:
sexo = "a"
ciudad = "Guanajuato"
c = "Bienvenid%s a %s, %.2f"%(sexo, ciudad, 3.141592654)
print(c)

Bienvenida a Guanajuato, 3.14


# Caminante aleatorio

In [28]:
p = figure()
caminante = pd.Series(np.random.randn(1000).cumsum())
p.line(caminante.index, caminante)
show(p)

In [36]:
test_stationarity(caminante, 100)

Resultados de la prueba de Dickey-Fuller:
Estadística de la prueba     -1.620014
valor-p                       0.472722
#retrasos usados              1.000000
Número de observaciones     998.000000
Valor crítico (1%)           -3.436919
Valor crítico (5%)           -2.864440
Valor crítico (10%)          -2.568314
dtype: float64


In [60]:
hn = heminorte-heminorte.shift(33)
hn.dropna(inplace=True)
p = figure(width=600, height=300)
p.line(hn.index, hn)
show(p)

In [59]:
test_stationarity(hn)

Resultados de la prueba de Dickey-Fuller:
Estadística de la prueba   -9.088360e+00
valor-p                     3.884515e-15
#retrasos usados            2.100000e+01
Número de observaciones     8.140000e+02
Valor crítico (1%)         -3.438409e+00
Valor crítico (5%)         -2.865097e+00
Valor crítico (10%)        -2.568664e+00
dtype: float64
